In [1]:
# ---- TP2 Reconstruccion 3D (versión corregida para Charuco dataset) ----
%load_ext autoreload
%autoreload 2

import glob, os
import numpy as np
import cv2
import cv2.aruco as aruco
from matplotlib import pyplot as plt
!pip install -qq git+https://github.com/udesa-vision/i308-utils.git
from i308_utils import imshow, show_images
import pickle

# -------------------- parámetros del tablero (ajustalos si hiciera falta) --------------------
# En tu caso confirmamos que el tablero físico tiene 6 columnas x 4 filas,
# pero probaste invertirlo — acá lo dejamos como vos definiste:
checkerboard_size = (4, 6)   # squaresX, squaresY  (probá invertir si no detecta)
square_size = 0.022          # metros
marker_size = 0.0165         # metros
dictionary = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)

# Crear CharucoBoard de forma compatible (distintas versiones de OpenCV)
if hasattr(aruco, "CharucoBoard_create"):
    board_charuco = aruco.CharucoBoard_create(
        squaresX=checkerboard_size[0],
        squaresY=checkerboard_size[1],
        squareLength=square_size,
        markerLength=marker_size,
        dictionary=dictionary
    )
else:
    board_charuco = aruco.CharucoBoard(
        checkerboard_size,
        square_size,
        marker_size,
        dictionary
    )

# Detector (se puede ajustar DetectorParameters si hace falta)
detector_params = aruco.DetectorParameters_create()
charuco_detector = aruco.CharucoDetector(board_charuco, detector_params)

# -------------------- ficheros --------------------
directory = os.path.join("datospropios", "calib")
left_files_pattern = "left_*.jpg"
right_files_pattern = "right_*.jpg"

def numeric_sort(file_name):
    base_name = os.path.basename(file_name)
    number_part = base_name.split("_")[-1].split(".")[0]
    try:
        return int(number_part)
    except ValueError:
        return -1

left_file_names = sorted(glob.glob(os.path.join(directory, left_files_pattern)), key=numeric_sort)
right_file_names = sorted(glob.glob(os.path.join(directory, right_files_pattern)), key=numeric_sort)

print("Buscando imágenes en:", os.path.abspath(directory))
print("Archivos left encontrados:", len(left_file_names))
print("Archivos right encontrados:", len(right_file_names))
print(left_file_names[:5])
print(right_file_names[:5])

if len(left_file_names) == 0 or len(right_file_names) == 0:
    raise SystemExit("No se encontraron imágenes. Ajustá la ruta/patrón.")

if len(left_file_names) != len(right_file_names):
    print("Atención: cantidad distinta de left/right, usaré zip y procesaré pares hasta el menor.")
# --------------------------------------------------------------------------------------------

# listas para stereoCalibrate
objectPoints_list = []   # lista de (N,3) por cada par válido
imgPoints_left = []      # lista de (N,1,2) por cada par válido
imgPoints_right = []

image_size = None
min_common_corners = 6   # criterio: mínimo de esquinas comunes entre ambos para aceptar el par

# función auxiliar: extrae puntos 2D en el orden de ids_requested
def _extract_corners_for_ids(all_ids, all_corners, ids_requested):
    # all_ids: (K,1), all_corners: list-like K x (1,2)
    mapping = {int(i[0]): idx for idx, i in enumerate(all_ids)}
    pts = []
    for idv in ids_requested:
        idx = mapping[int(idv)]
        # all_corners[idx] es array shape (1,2); tomamos la tupla (x,y)
        pts.append(all_corners[idx][0])
    return np.array(pts, dtype=np.float32)  # shape (M,2)

# recorremos pares
for li, ri in zip(left_file_names, right_file_names):
    print("processing", li, ri)
    imgL_color = cv2.imread(li, cv2.IMREAD_COLOR)
    imgR_color = cv2.imread(ri, cv2.IMREAD_COLOR)
    if imgL_color is None or imgR_color is None:
        print("  Warning: no se pudo leer alguna imagen, saltando.")
        continue

    grayL = cv2.cvtColor(imgL_color, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR_color, cv2.COLOR_BGR2GRAY)

    if image_size is None:
        image_size = (grayL.shape[1], grayL.shape[0])

    # 1) detectar marcadores ArUco en cada imagen
    cornersL, idsL, rejectedL = aruco.detectMarkers(grayL, dictionary, parameters=detector_params)
    cornersR, idsR, rejectedR = aruco.detectMarkers(grayR, dictionary, parameters=detector_params)
    print(f"  Marcadores detectados L={0 if idsL is None else len(idsL)}  R={0 if idsR is None else len(idsR)}")

    # visualizar detecciones (opcional)
    # visL = aruco.drawDetectedMarkers(imgL_color.copy(), cornersL, idsL)
    # visR = aruco.drawDetectedMarkers(imgR_color.copy(), cornersR, idsR)
    # show_images([visL, visR], ["L markers", "R markers"])

    if idsL is None or idsR is None:
        print("  -> 0 marcadores en alguna vista, saltando.")
        continue

    # 2) interpolar esquinas Charuco (necesita que detectMarkers haya encontrado algunos)
    # devuelve retval (número corners interpolados), charucoCorners (Nx1x2), charucoIds (Nx1)
    retvalL, charucoCornersL, charucoIdsL = aruco.interpolateCornersCharuco(
        markerCorners=cornersL, markerIds=idsL, image=grayL, board=board_charuco
    )
    retvalR, charucoCornersR, charucoIdsR = aruco.interpolateCornersCharuco(
        markerCorners=cornersR, markerIds=idsR, image=grayR, board=board_charuco
    )
    nL = 0 if charucoIdsL is None else len(charucoIdsL)
    nR = 0 if charucoIdsR is None else len(charucoIdsR)
    print(f"  Charuco interpolated L={nL}  R={nR}")

    if charucoIdsL is None or charucoIdsR is None:
        print("  -> no se pudieron interpolar esquinas Charuco en alguna vista, saltando.")
        continue

    # 3) quedarnos solo con IDs comunes detectados en L y R (mismo orden)
    common_ids = np.intersect1d(charucoIdsL.flatten(), charucoIdsR.flatten())
    print("  Common ids count:", len(common_ids))

    if len(common_ids) < min_common_corners:
        print(f"  -> menos de {min_common_corners} esquinas comunes, saltando.")
        continue

    common_ids_sorted = np.sort(common_ids)

    # extraer los puntos 2D correspondientes en ambas imágenes en el orden de common_ids_sorted
    left_2d = _extract_corners_for_ids(charucoIdsL, charucoCornersL, common_ids_sorted)   # (M,2)
    right_2d = _extract_corners_for_ids(charucoIdsR, charucoCornersR, common_ids_sorted)  # (M,2)

    # obtener object points 3D en el mismo orden (board_charuco.chessboardCorners está en coordenadas del tablero)
    # board_charuco.chessboardCorners es array (num_corners_total, 3) en unidades del squareLength
    # cada id en common_ids_sorted indexa a ese arreglo
    obj_pts = board_charuco.chessboardCorners[common_ids_sorted].astype(np.float32)  # (M,3)

    # re-dar forma a lo que espera OpenCV: imagePoints como (M,1,2), objectPoints como (M,3)
    imgL_for_cv = left_2d.reshape(-1, 1, 2)
    imgR_for_cv = right_2d.reshape(-1, 1, 2)

    objectPoints_list.append(obj_pts)   # tipo: array (M,3)
    imgPoints_left.append(imgL_for_cv)  # tipo: array (M,1,2)
    imgPoints_right.append(imgR_for_cv)

    left_images.append(grayL)
    right_images.append(grayR)

    print(f"  -> Aceptado par: {li} / {ri} con {len(common_ids_sorted)} esquinas comunes.")

print("Total pares válidos recolectados:", len(objectPoints_list))

# -------------------- calibración estéreo (si tenemos suficientes pares) --------------------
if len(objectPoints_list) < 8:
    print("No hay suficientes pares para una calibración estéreo robusta (recomendado >=8).")
else:
    print("Ejecutando cv2.stereoCalibrate ...")
    # convert to lists as OpenCV expects Python lists
    objPts = [op.astype(np.float32) for op in objectPoints_list]
    imgPtsL = [ip.astype(np.float32) for ip in imgPoints_left]
    imgPtsR = [ip.astype(np.float32) for ip in imgPoints_right]

    # iniciales: None -> OpenCV estima intrínsecos también
    flags = 0
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)

    ret, M1, D1, M2, D2, R, T, E, F = cv2.stereoCalibrate(
        objectPoints=objPts,
        imagePoints1=imgPtsL,
        imagePoints2=imgPtsR,
        cameraMatrix1=None,
        distCoeffs1=None,
        cameraMatrix2=None,
        distCoeffs2=None,
        imageSize=image_size,
        flags=flags,
        criteria=criteria
    )

    print("stereoCalibrate ret:", ret)
    print("M1:\n", M1)
    print("D1:\n", D1.ravel())
    print("M2:\n", M2)
    print("D2:\n", D2.ravel())
    print("R:\n", R)
    print("T:\n", T)
    print("E:\n", E)
    print("F:\n", F)

    # guardar resultados
    calibration_results = {
        'M1': M1, 'D1': D1, 'M2': M2, 'D2': D2, 'R': R, 'T': T, 'E': E, 'F': F, 'image_size': image_size
    }
    save_path = os.path.join("code_examples", "charuco_example", "res")
    os.makedirs(save_path, exist_ok=True)
    with open(os.path.join(save_path, "stereo_calibration_from_charuco.pkl"), "wb") as f:
        pickle.dump(calibration_results, f)
    print("Resultados guardados en", os.path.join(save_path, "stereo_calibration_from_charuco.pkl"))


AttributeError: module 'cv2.aruco' has no attribute 'DetectorParameters_create'